In [ ]:
#--------Loading in Packages------#
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from scipy.io import mmread
from scipy.sparse import csr_matrix
import seaborn as sns
import os
import tacco as tc

In [ ]:
#------Loading in 8um bin filtered dataset-------#
adata = sc.read_h5ad("/projects/b1217/Edward/Python_Projects/HHA/ELfullcdata/ELfull_sdata8um-f100.h5ad")
adata

In [ ]:
#--------Loading in Full scRNA Atlas-----#
HHA = sc.read_h5ad("/projects/b1217/Edward/R_Projects/HHA/h5ad/20250516_HHA-SC-Final.h5ad")
HHA

In [ ]:
#--------Plotting UMAPs------#
#General Annotation
sc.pl.umap(HHA, color="GeneralAnnotation", frameon=False, legend_loc="on data")
#Final Annotation
sc.pl.umap(HHA, color="FinalAnnotation", frameon=False, legend_loc="on data")

In [ ]:
#------Loading in IFE Data from Seurat------#
ConversionDirectory =  "/projects/b1217/HHA/Multiome_Scanpy_Conversion/Full_Atlas/"
#Reading in the count matrix from seurat.combined
counts = mmread(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_IFE_Conversion_RawCounts_4_1_25.mtx"))
counts = csr_matrix(counts.transpose()) #transposing to cell x gene matrix and converting to csr format
#Contains metadata, barcodes, and umap embeddings 
metadata = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_IFE_Conversion_Metadata_4_1_25.csv"))
#Contains gene names 
gene_names = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_IFE_Conversion_GeneNames_4_1_25.csv"))
#Converting to anndata object
IFE = ad.AnnData(counts) #creating anndata object
IFE.obs_names = metadata["barcode"] #Adding barcodes as obs names 
IFE.var_names = gene_names["gene"] #adding gene names 
print(IFE)
print(IFE.obs_names[:10])
print(IFE.var_names[:10])
scvi = pd.read_csv(os.path.join(ConversionDirectory, "HHA_SCRNA_Multiome_IFE_Conversion_scVI_Embeddings_4_1_25.csv"))
scvi["barcode"] = metadata["barcode"]
#------Adding metadata to anndata--------#
IFE.obs = metadata
IFE.obs_names = metadata["barcode"]

#-----Adding dimensionality reductions to obsm-----#
X_umap = metadata.set_index("barcode")[["UMAP_1", "UMAP_2"]]
IFE.obsm['X_umap'] = X_umap.loc[IFE.obs_names].values
X_scvi = scvi.set_index("barcode")
IFE.obsm['X_scvi'] = X_scvi.loc[IFE.obs_names].values
print(IFE.obs.head())

In [ ]:
#-----Plotting Clusters and Genes------#
#Dictionary mapping colors to each cluster
color_dict = {"Basal": "#2E8289FF",
             "Spinous": "#B4674EFF",
             "Early_Granular": "#EAAE37FF",
             "Late_Granular": "#565F41FF"}
#Clusters
sc.pl.umap(IFE,
    color="IFEAnnotation",
    frameon=False,
    add_outline=True,
    palette = color_dict,
    size = 15)

In [ ]:
#-------Converting gene expression matrices to float for TACCO------#
#Switching to raw count matrix
HHA.X = HHA.layers["counts_RNA"]
HHA.X = HHA.X.astype(np.float64)
#Already raw count matrix
adata.X = adata.X.astype(np.float64)
IFE.X = IFE.X.astype(np.float64)

In [ ]:
#-----Crossmapping var_names to EnsemblID--------#
IFE.var["gene"] = IFE.var_names
symbol_to_ensembl = dict(zip(HHA.var["names"], HHA.var["ensembl"]))
def map_to_ensembl(name):
    if name.startswith("ENSG"):
        return name
    return symbol_to_ensembl.get(name, "Unknown")
IFE.var["ensembl"] = IFE.var["gene"].apply(map_to_ensembl)

#-------Creating EnsemblID column and set as index--------#
IFE = IFE[:, IFE.var["ensembl"] != "Unknown"].copy()
new_index = IFE.var["ensembl"]
IFE.var.index = new_index
IFE.var_names = new_index
IFE.var

In [ ]:
#------Round 1 Mapping: Running TACCO on Full Dataset-------#
tc.tl.annotate(adata, HHA, annotation_key='FinalAnnotation', result_key='FinalAnnotation')

#-----Retrieving Best Annotation Mapping for Each Cluster------#
tc.utils.get_maximum_annotation(adata, 'FinalAnnotation', result_key='FinalAnnotation')
adata

In [ ]:
#-------Plotting Global TACCO Results-----#
#Pulling Palette from scRNA adata
FinalAnnotation_palette = dict(zip(HHA.obs["FinalAnnotation"].cat.categories, HHA.uns["FinalAnnotation_colors"]))
FinalAnnotation_palette

#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(adata, color=["FinalAnnotation"], img_key="hires", ax=ax, palette = FinalAnnotation_palette)

In [ ]:
#--------------Filtering Spatial Data for IFE Spots----------#
IFE_Spatial = adata[adata.obs["FinalAnnotation"].isin(["IFE_Basal", "IFE_Suprabasal"]),:]
IFE_Spatial

In [ ]:
#------Second Iteration TACCO run on Spots Mapping to Melanocytes------#
tc.tl.annotate(IFE_Spatial, IFE, annotation_key='IFEAnnotation', result_key='IFEAnnotation')

#-----Retrieving Best Annotation Mapping for Each Cluster------#
tc.utils.get_maximum_annotation(IFE_Spatial, 'IFEAnnotation', result_key='IFEAnnotation')

IFE_Spatial

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(40, 40))
sc.pl.spatial(IFE_Spatial, color=["IFEAnnotation"], img_key="hires", ax=ax, palette = color_dict)

In [ ]:
#---------------Pulling Scores from Obsm----------------#
Annot_Scores = IFE_Spatial.obsm['IFEAnnotation'].copy()
Annot_Names = [x + "_Score" for x in Annot_Scores.columns]
Annot_Scores.columns = Annot_Names
Annot_Scores
#Adding to obs
for Score in Annot_Scores.columns:
    IFE_Spatial.obs[Score] = Annot_Scores[Score]
IFE_Spatial.obs

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(IFE_Spatial, color=["Basal_Score"], img_key="hires", ax=ax, cmap = 'magma')

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(IFE_Spatial, color=["Spinous_Score"], img_key="hires", ax=ax, cmap = 'magma')

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(IFE_Spatial, color=["Early_Granular_Score"], img_key="hires", ax=ax, cmap = 'magma')

In [ ]:
#--------Plotting Full Image-------#
fig, ax = plt.subplots(figsize=(20, 20))
sc.pl.spatial(IFE_Spatial, color=["Late_Granular_Score"], img_key="hires", ax=ax, cmap = 'magma')

In [ ]:
#--------Saving Annotated Objects---------#
#Full Mapping
IFE_Spatial.write_h5ad('/projects/b1217/HHA/IFE_Spatial_Mapping/IFE_Spatial_Annotated_1_14_26.h5ad')

In [ ]:
#--------Saving IFE SCRNA Object------------#
del IFE.obs["Likely_Ruptured_075"]
IFE.write_h5ad('/projects/b1217/HHA/IFE_Spatial_Mapping/IFE_1_14_26.h5ad')